In [25]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:

from buttermilk import BM
from promptflow.tracing import trace, start_trace
from rich import print as rprint

import os

from hydra import initialize, compose
from omegaconf import OmegaConf

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="./conf"):
    cfg = compose(config_name='config',
                  overrides=["+data=trans", "+step=trans", "+save=bq", "job=score","source=notebook"])

# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)

# Print config
rprint(OmegaConf.to_container(bm.cfg, resolve=True))



2024-11-07 15:18:01 J5HW6L4KT6 buttermilk buttermilk.py[ 182] DEBUG Config passed in but we already have one loaded. Overwriting.


DEBUG:buttermilk:Config passed in but we already have one loaded. Overwriting.


{
    'name': 'automod',
    'job': 'score',
    'source': 'notebook',
    'secret_provider': 'azure',
    'logger': 'gcp',
    'tracing': True,
    'verbose': True,
    'models_secret': 'models',
    'save_dest': 'gcp',
    'save_dir': None,
    'gcp': {'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'},
    'azure': {'vault': 'https://suzorvault.vault.azure.net/', 'resource_group': 'rg-suzor_ai'},
    'aws': {'secret_name': 'secret', 'region': 'us-east-1'},
    'step': [
        {
            'flow': 'judger',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc',
                'template': 'judge',
                'criteria': [
                    'trans_cte',
                    'trans_tja',
                    'trans_glaad',
                    'trans_hrc',
                    'trans_simplified',
                    'trans_factored'
                ],
                'formatting': 'json_rules',
                'model': ['haiku', 'llama31_70b', 'gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'record_id': 'record_id', 'content': 'content', 'title': 'title'},
            'data': None
        },
        {
            'flow': 'synth',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc_judge',
                'template': 'synthesise',
                'criteria': ['trans_simplified'],
                'formatting': 'json_rules',
                'model': ['haiku', 'gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'answers': 'answers', 'meta': 'meta'},
            'data': {
                'judger': {
                    'name': 'previous runs',
                    'type': 'job',
                    'dataset': 'dmrc-analysis.toxicity.flow',
                    'max_records_per_group': 1,
                    'filter': {'agent_info.flow': 'judger', 'outputs.reasons': ['NOTNULL']},
                    'join': {'record_id': 'record.record_id'},
                    'group': {
                        'flow': 'agent_info.flow',
                        'template': 'agent_info.template',
                        'criteria': 'agent_info.template_vars.criteria',
                        'model': 'parameters.model'
                    },
                    'columns': {
                        'answers': {
                            'id': 'job_id',
                            'model': 'model',
                            'criteria': 'criteria',
                            'template': 'template',
                            'flow': 'flow',
                            'reasons': 'outputs.reasons'
                        },
                        'meta': {
                            'flow': 'agent_info.flow',
                            'template': 'agent_info.template',
                            'criteria': 'agent_info.template_vars.criteria',
                            'model': 'parameters.model',
                            'prediction': 'outputs.prediction',
                            'labels': 'outputs.labels',
                            'confidence': 'outputs.confidence',
                            'timestamp': 'timestamp'
                        }
                    }
                }
            }
        },
        {
            'flow': 'eval',
            'num_runs': 1,
            'concurrent': 20,
            'agent': {
                'name': 'lc',
                'template': 'differences',
                'criteria': ['trans_simplified'],
                'model': ['haiku', 'gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'answers': 'answers', 'meta': 'meta'},
            'data': {
                'judger': {
                    'name': 'previous runs and synth',
                    'type': 'job',
                    'dataset': 'dmrc-analysis.toxicity.flow',
                    'max_records_per_group': 1,


## Check results at judge and synth stages

In [44]:
from buttermilk.runner.helpers import prepare_step_data, load_data

df = prepare_step_data(cfg.data, cfg.step[3].data)
df.info()


2024-11-07 15:18:10 J5HW6L4KT6 buttermilk buttermilk.py[ 376] INFO Query stats: Ran in 0:00:04.061176 seconds, cache hit: False, billed 28.31 MB, approx cost $1.4e-05.


INFO:buttermilk:Query stats: Ran in 0:00:04.061176 seconds, cache hit: False, billed 28.31 MB, approx cost $1.4e-05.


<class 'pandas.core.frame.DataFrame'>
Index: 9 entries, 8 to 5
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   record_id   9 non-null      object
 1   content     9 non-null      object
 2   title       9 non-null      object
 3   model       9 non-null      object
 4   confidence  9 non-null      object
 5   criteria    9 non-null      object
 6   prediction  9 non-null      object
 7   template    9 non-null      object
 8   reasons     9 non-null      object
 9   timestamp   9 non-null      object
 10  labels      9 non-null      object
dtypes: object(11)
memory usage: 864.0+ bytes


In [28]:
df.reasons


AttributeError: 'DataFrame' object has no attribute 'reasons'

In [19]:
import pandas as pd
import json
df


,record_id,record_id_x,content,title,job_id,timestamp,run_info,agent_info,source,record,parameters,outputs,error,metadata,prompt,record_id_y
5,transgender_woman_murder_Baltimore,transgender_woman_murder_Baltimore,Tai’Vion Lathan had an infectious laugh that m...,A transgender woman was found killed in a West...,"[83Ka5kU2vXRXGiM5jE5cTz, 7iAU2nihU5BEwghbxo3DP...","[2024-11-06 01:18:59.266121+00:00, 2024-11-06 ...","[{""ip"":""159.196.210.27"",""job"":""score"",""node_na...","[{""agent"":""lc_9ue3pE"",""flow"":""judger"",""model"":...","[[notebook], [notebook], [notebook], [notebook...",[{'content': 'Tai’Vion Lathan had an infectiou...,"[{""connection"":{""configs"":{""model"":""gemini-1.5...","[{""confidence"":""high"",""labels"":[],""prediction""...","[None, None, None, None, None, None, None, Non...","[{""outputs"":{""citation_metadata"":{""citations"":...",[[system:\nYou are a careful and precise audit...,"[transgender_woman_murder_Baltimore, transgend..."
7,dylan_thomas_murder,dylan_thomas_murder,"Dylan Gurley, a 20-year-old transgender woman,...",20-year-old transgender woman Dylan Gurley sta...,"[frMzvDB7QmgVuQjkdM59Zn, TYNPp9BfiJJDBBG9PQHZk...","[2024-11-06 01:20:03.831353+00:00, 2024-11-06 ...","[{""ip"":""159.196.210.27"",""job"":""score"",""node_na...","[{""agent"":""lc_6kcnfX"",""flow"":""judger"",""model"":...","[[notebook], [notebook], [notebook], [notebook...","[{'content': 'Dylan Gurley, a 20-year-old tran...","[{""connection"":{""configs"":{""do_sample"":true,""e...","[{""confidence"":""high"",""labels"":[],""prediction""...","[None, None, None, None, None, None, None, Non...","[{""outputs"":{""finish_reason"":""stop"",""seconds_e...",[[system:\nYou are a careful and precise audit...,"[dylan_thomas_murder, dylan_thomas_murder, dyl..."
4,NCAA_transgender_athletes,NCAA_transgender_athletes,"Like any competitive swimmer, Meghan Cortez-Fi...",A college athletics association banned transge...,"[H3UahgNust3mX4gQ2kaPSt, USfxaTSpEdAnu23wiFfF5...","[2024-11-06 01:18:59.889885+00:00, 2024-11-05 ...","[{""ip"":""159.196.210.27"",""job"":""score"",""node_na...","[{""agent"":""lc_9ue3pE"",""flow"":""judger"",""model"":...","[[notebook], [notebook], [notebook], [notebook...","[{'content': 'Like any competitive swimmer, Me...","[{""connection"":{""configs"":{""model"":""gemini-1.5...","[{""confidence"":""high"",""labels"":[],""prediction""...","[None, None, None, None, None, None, None, Non...","[{""outputs"":{""finish_reason"":""STOP"",""is_blocke...",[[system:\nYou are a careful and precise audit...,"[NCAA_transgender_athletes, NCAA_transgender_a..."
6,agony_of_page,agony_of_page,"Last year, Ellen Page—now going by ‘Elliot’ an...",The Agony of Ellen Page,"[NBcysBWefCfYBrZvmNvrLK, W3LD6ddLCsEjjXMxbwZEx...","[2024-11-06 01:20:50.721178+00:00, 2024-11-06 ...","[{""ip"":""159.196.210.27"",""job"":""score"",""node_na...","[{""agent"":""lc_coh2D6"",""flow"":""judger"",""model"":...","[[notebook], [notebook], [notebook], [notebook...","[{'content': 'Last year, Ellen Page—now going ...","[{""connection"":{""configs"":{""do_sample"":true,""e...","[{""confidence"":""medium"",""labels"":[""Transphobic...","[None, None, None, None, None, None, None, Non...","[{""outputs"":{""finish_reason"":""stop"",""seconds_e...",[[system:\nYou are a careful and precise audit...,"[agony_of_page, agony_of_page, agony_of_page, ..."
0,kansas_highway,kansas_highway,"TOPEKA, Kan. -- Kansas will pay $50,000 to set...","Kansas will pay $50,000 to settle a suit over ...","[Bui3APtqSMeV6Gy4o2v4Gn, dxFeXFd8b23dkKuyt9ASr...","[2024-11-05 08:20:03.882071+00:00, 2024-11-06 ...","[{""ip"":""159.196.210.27"",""job"":""score"",""node_na...","[{""agent"":""lc_f9fH8m"",""flow"":""judger"",""model"":...","[[notebook], [notebook], [notebook], [notebook...","[{'content': 'TOPEKA, Kan. -- Kansas will pay ...","[{""connection"":{""configs"":{""model"":""claude-3-h...","[{""confidence"":""high"",""labels"":[],""prediction""...","[None, None, None, None, None, None, None, Non

In [ ]:
class MyConfig(BaseModel):
    name: str
    age: int

Config = builds(MyConfig, populate_full_signature=True)

@hydra.main(config_path="conf", config_name="config")
def my_app(cfg: DictConfig) -> None:
    validated_config = instantiate(Config, cfg)
    print(validated_config)


In [ ]:
from hydra_zen import store
from hydra_zen.third_party.pydantic import pydantic_parser

from typing import Literal
from pydantic import PositiveInt
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
from hydra_zen import builds
ZenBuilds_MFO = builds(MultiFlowOrchestrator, populate_full_signature=True)
ZenBuilds_MFO

from hydra_zen import zen

store(main, hydra_defaults=["_self_", {"character": "base"}])
store(Character, group="character", name="base")
store.add_to_hydra_store()

zen(
    main,
    # This is the key ingredient
    instantiation_wrapper=pydantic_parser,
).hydra_main(
    config_name="main",
    config_path=None,
    version_base="1.3",
)


types.Builds_MultiFlowOrchestrator

## Second step is a synthesis of previous draft answers

In [5]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[1], data=cfg.data, save=cfg.save, source=cfg.source)


2024-11-06 12:33:39 J5HW6L4KT6 buttermilk buttermilk.py[ 376] INFO Query stats: Ran in 0:00:06.161060 seconds, cache hit: False, billed 15.73 MB, approx cost $7.9e-06.


INFO:buttermilk:Query stats: Ran in 0:00:06.161060 seconds, cache hit: False, billed 15.73 MB, approx cost $7.9e-06.


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 350 entries, ('transgender_paralympian_dailymail', 'judger', 'judge', 'trans_tja', 'llama31_70b') to ('transgender_paralympian_BBC', 'judger', 'judge', 'trans_hrc', 'gpt4o')
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          350 non-null    object             
 1   timestamp   350 non-null    datetime64[us, UTC]
 2   run_info    350 non-null    object             
 3   agent_info  350 non-null    object             
 4   source      350 non-null    object             
 5   record      350 non-null    object             
 6   parameters  350 non-null    object             
 7   outputs     350 non-null    object             
 8   error       0 non-null      object             
 9   metadata    350 non-null    object             
 10  prompt      350 non-null    object             
 11  record_id   350 non-null    object             
 

In [6]:
# Inspect the dataset so we can make sure the inputs look ok
rprint(orchestrator._dataset.sample().answers.values)


[list([{'id': 'J6SqUtdwyQX5uuLk5hWCYa', 'model': 'gemini15pro', 'criteria': 'trans_cte', 'template': 'judge', 
'flow': 'judger', 'reasons': ["The article discusses the murder of Tai'Vion Lathan, a Black transgender woman, 
highlighting the increased violence faced by the transgender community, particularly transgender women of color.  
It uses Lathan's correct name and pronouns throughout the piece. The article aims to raise awareness about this 
issue and advocate for justice and policy change to protect transgender individuals. The article's tone is 
respectful and aims to humanize Lathan, emphasizing her personality, family, and aspirations.", "The content 
consistently uses Tai'Vion Lathan's chosen name and pronouns (she/her) throughout the article, as requested by her 
family. This aligns with the provided guidelines from the Associated Press Stylebook and the New York Times Style 
Book.", 'The article specifically discusses the violence faced by transgender people, especially transgender women 
of color. It highlights the statistics on transgender homicides and discusses the broader context of legislative 
attacks on transgender rights and healthcare. The language used is respectful and aims to raise awareness of the 
issue.', "The article focuses on the tragic murder of a transgender woman and the broader issues faced by the 
transgender community. By highlighting Lathan's personal life, aspirations, and the grief of her family, the 
article humanizes her and counters the dehumanization often experienced by transgender individuals.", 'The article 
promotes respect and understanding of transgender people by using accurate terminology and highlighting the 
importance of recognizing their full humanity, beyond their gender identity. It also advocates for justice and 
policy change to protect transgender individuals from violence and discrimination.']}, {'id': 
'Qgi5p3fnPShknhHsTNBf2X', 'model': 'gpt4o', 'criteria': 'trans_cte', 'template': 'judge', 'flow': 'judger', 
'reasons': ['The content is a news report covering the death of Tai’Vion Lathan, a transgender woman, who was 
murdered in Baltimore. It discusses the circumstances of her death, the grief of her family, and provides 
statistics on violence against transgender individuals, especially Black transgender women.', "According to the 
guidelines, the pronouns and names used should be those preferred by the individual. The content uses 'Tai’Vion 
Lathan,' also known as 'Tai' and uses 'her' and 'she' pronouns throughout, indicating adherence to this guideline 
as there are no signs to contrary regarding Lathan's preferred pronouns.", "The content does cite Tai’Vion Lathan's
transgender status, but it is pertinent to the context, as the article addresses violence against transgender 
individuals and highlights ongoing societal issues faced by this community. Therefore, this mention is justified as
it provides context that is necessary for understanding the motive of the crime and its broader social 
implications.", "According to the guidelines, terminology related to transgender individuals should be used 
sensitively and accurately. Terms like 'transgender woman' are used in the article, which aligns correctly with 
guideline terminology.", "The article provides national context concerning issues faced by transgender people, such
as violence and anti-trans legislation, which align with key issues identified within the guidelines. It 
sensitively incorporates this information to elevate public awareness of the broader context in which Tai’Vion 
Lathan's murder occurred."]}, {'id': 'h2yUoJ6ii2YUwdJ5cgvF8z', 'model': 'haiku', 'criteria': 'trans_cte', 
'template': 'judge', 'flow': 'judger', 'reasons': ["The content provided is a news article reporting on the tragic 
murder of a 24-year-old transgender woman named Tai'Vion Lathan in Baltimore, Maryland. The article provides 
details about Lathan's life, the circumstances of her death, and the broader context of violence fa

In [8]:
await orchestrator.run_tasks()


2024-11-06 12:38:44 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_judge_YNpFkj with job ZoRBmAyKKxiPZhU4CyYZ76.


DEBUG:buttermilk:Starting task for Agent lc_judge_YNpFkj with job ZoRBmAyKKxiPZhU4CyYZ76.


2024-11-06 12:38:44 J5HW6L4KT6 buttermilk templating.py[  50] DEBUG Unable to decode template as Prompty: Illegal formatting of prompty. The prompt file is in markdown format and can be divided into two parts, the first part is in YAML format and contains connection and model information. The second part is the prompt template., e.args=('Illegal formatting of prompty. The prompt file is in markdown format and can be divided into two parts, the first part is in YAML format and contains connection and model information. The second part is the prompt template.',)


DEBUG:buttermilk:Unable to decode template as Prompty: Illegal formatting of prompty. The prompt file is in markdown format and can be divided into two parts, the first part is in YAML format and contains connection and model information. The second part is the prompt template., e.args=('Illegal formatting of prompty. The prompt file is in markdown format and can be divided into two parts, the first part is in YAML format and contains connection and model information. The second part is the prompt template.',)


2024-11-06 12:38:44 J5HW6L4KT6 buttermilk lc.py[ 204] DEBUG Invoking chain with haiku...


...

DEBUG:buttermilk:Agent lc_judge_YNpFkj completed job ZoRBmAyKKxiPZhU4CyYZ76 successfully.


2024-11-06 12:49:15 J5HW6L4KT6 buttermilk orchestrator.py[ 106] INFO Completed run 1 of 1


INFO:buttermilk:Completed run 1 of 1


2024-11-06 12:49:15 J5HW6L4KT6 buttermilk orchestrator.py[ 110] INFO All tasks have completed.


INFO:buttermilk:All tasks have completed.


## Third step, we summarise the answers, similarities, and differences

In [9]:
# Create an orchestrator to conduct all combinations of jobs we want to run
from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(step=cfg.step[2], data=cfg.data, save=cfg.save, source=cfg.source)


2024-11-06 15:08:06 J5HW6L4KT6 buttermilk buttermilk.py[ 376] INFO Query stats: Ran in 0:00:03.769667 seconds, cache hit: False, billed 22.02 MB, approx cost $1.1e-05.


INFO:buttermilk:Query stats: Ran in 0:00:03.769667 seconds, cache hit: False, billed 22.02 MB, approx cost $1.1e-05.


In [10]:
await orchestrator.run_tasks()


2024-11-06 15:09:03 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_H2Cy74 with job ZCss8xW3KHbnFdkAJ3hUoN.


DEBUG:buttermilk:Starting task for Agent lc_H2Cy74 with job ZCss8xW3KHbnFdkAJ3hUoN.


2024-11-06 15:09:03 J5HW6L4KT6 buttermilk lc.py[ 204] DEBUG Invoking chain with haiku...


DEBUG:buttermilk:Invoking chain with haiku...


2024-11-06 15:09:04 J5HW6L4KT6 buttermilk orchestrator.py[  81] DEBUG Starting task for Agent lc_H2Cy74 with job dLzUUufZDsER7RGHeYAXb7.


...

DEBUG:buttermilk:Agent lc_h5QtST completed job HSdm245NueuhHwmYTA9iCC successfully.


2024-11-06 15:11:21 J5HW6L4KT6 buttermilk orchestrator.py[ 106] INFO Completed run 1 of 1


INFO:buttermilk:Completed run 1 of 1


2024-11-06 15:11:21 J5HW6L4KT6 buttermilk orchestrator.py[ 110] INFO All tasks have completed.


INFO:buttermilk:All tasks have completed.
